In [19]:
import numpy as np
import pandas as pd
import datetime as dt
import dash
import plotly.express as px
import dash_html_components as html
import dash_core_components as dcc
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output
from jupyter_dash import JupyterDash

In [95]:
Stats = pd.read_csv("Stats.csv", sep=";")
Summary = pd.read_csv("Summary.csv", sep=";")
Teams = pd.read_csv("Teams.csv", sep= ";")
Players = pd.read_csv("PlayerList.csv", sep= ";")

In [96]:
#DATA VISUALIZATION WITH PLOTLY
playersPositionFig = px.pie(data_frame=Players, names="Pozycja")

In [185]:
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
controls = dbc.Card(
    [
        dbc.FormGroup(
            [
                dbc.Label("Gdzie"),
                dcc.Dropdown(id="Where", 
                options=[{'label': "Dom", "value":"Dom"},
                         {'label': "Wyjazd", "value":"Wyjazd"},
                         {'label': "Oba", "value":"Oba"}
                        ],
                value= "Dom"),
                
            ]
        ),
        dbc.FormGroup(
            [
                dbc.Label("Bramki"),
                dcc.Dropdown(id="Score", 
                options=[{'label': "Stracone", "value":"Stracone"},
                         {'label': "Zdobyte", "value":"Zdobyte"},
                         {'label': "Bilans", "value":"Oba"}
                        ],
                value= "Zdobyte"),
                
            ]
        ),
        dbc.FormGroup(
            [
                dbc.Label("Kolejki"),
                dcc.RangeSlider(
                    id='Kolejki',
                    min=1,
                    max=30,
                    step=1,
                    value=[1, 30]
                ),
                html.Div(id='output-container-range-slider'),   
            ]
        ),
    ],
    body = True,
)
app.layout = dbc.Container(
    [
        html.H1("Liczba zdobytych bramek w sezonie"),
        html.Hr(),
        dbc.Row(
            [
                dbc.Col(controls, md=4),
                dbc.Col(dcc.Graph(id='histogram', figure = px.histogram(data_frame=Stats, x = "Zespol1-Nazwa",
                                                   y = "Zespol1-Bramki")), md=8),
            ],
            align="center",
        ),
    ],
    fluid=True,
)

@app.callback(
    Output('output-container-range-slider', 'children'),
    Output(component_id="histogram", component_property="figure"),
    [
        Input('Kolejki', 'value'),
        Input(component_id="Where", component_property="value"),
        Input(component_id="Score", component_property="value"),
    ],
)
def CreateDashboard(Kolejki, Where, Score):
    who1 = ''
    score1 = ''
    #add description
    description = "Kolejki od " + str(Kolejki[0]) + " do " + str(Kolejki[1])
    #narrows rounds
    StatsForDash = Stats[(Stats["Kolejka"] >=Kolejki[0]) & (Stats["Kolejka"] <=Kolejki[1])]
    figure = px.histogram()
    #Wyjazd
    if Where == "Wyjazd":
        #Zdobyte
        if Score == "Zdobyte":
            figure = px.histogram(data_frame=StatsForDash, x = "Zespol2-Nazwa", y = "Zespol2-Bramki").update_xaxes(
                categoryorder='total descending')
        #Stracone
        elif Score == "Stracone":
        #Bilans wyjazd
            figure = px.histogram(data_frame=StatsForDash, x = "Zespol2-Nazwa", y = "Zespol1-Bramki").update_xaxes(
                categoryorder='total descending')
        else:
            Goals1 = StatsForDash[["Zespol2-Nazwa", "Zespol2-Bramki"]].groupby(['Zespol2-Nazwa']).agg('sum').reset_index()
            Goals2 = StatsForDash[["Zespol2-Nazwa", "Zespol1-Bramki"]].groupby(['Zespol2-Nazwa']).agg('sum').reset_index()
            Goals1["Zespol1-Bramki"] = Goals1["Zespol2-Bramki"] - Goals2["Zespol1-Bramki"]
            figure = px.histogram(data_frame=Goals1, x = "Zespol2-Nazwa", y = "Zespol1-Bramki").update_xaxes(
                categoryorder='total descending')
    #Dom
    elif Where == "Dom":
        #Zdobyte
        if Score == "Zdobyte":
            figure = px.histogram(data_frame=StatsForDash, x = "Zespol1-Nazwa", y = "Zespol1-Bramki").update_xaxes(
                categoryorder='total descending')
            #Stracone
        elif Score == "Stracone":
            figure = px.histogram(data_frame=StatsForDash, x = "Zespol1-Nazwa", y = "Zespol2-Bramki").update_xaxes(
                categoryorder='total descending')
        #Bilans Dom
        else:
            Goals1 = StatsForDash[["Zespol1-Nazwa", "Zespol1-Bramki"]].groupby(['Zespol1-Nazwa']).agg('sum').reset_index()
            Goals2 = StatsForDash[["Zespol1-Nazwa", "Zespol2-Bramki"]].groupby(['Zespol1-Nazwa']).agg('sum').reset_index()
            Goals1["Zespol1-Bramki"] = Goals1["Zespol1-Bramki"] - Goals2["Zespol2-Bramki"]
            figure = px.histogram(data_frame=Goals1, x = "Zespol1-Nazwa", y = "Zespol1-Bramki").update_xaxes(
                categoryorder='total descending')
    #Dom + Wyjazd
    else:
        #Zdobyte
        if Score == "Zdobyte":
            Goals1 = StatsForDash[["Zespol1-Nazwa", "Zespol1-Bramki"]].groupby(['Zespol1-Nazwa']).agg('sum').reset_index()
            Goals2 = StatsForDash[["Zespol2-Nazwa", "Zespol2-Bramki"]].groupby(['Zespol2-Nazwa']).agg('sum').reset_index()
            Goals2.columns = ["Zespol1-Nazwa", "Zespol1-Bramki"]
            Goals1 = Goals1.append(Goals2, ignore_index=True)
            Goals1 = Goals1.groupby(['Zespol1-Nazwa']).agg('sum').reset_index()
            figure = px.histogram(data_frame=Goals1, x = "Zespol1-Nazwa", y = "Zespol1-Bramki").update_xaxes(
                categoryorder='total descending')
        #Stracone
        elif Score == "Stracone":
            Goals1 = StatsForDash[["Zespol1-Nazwa", "Zespol2-Bramki"]].groupby(['Zespol1-Nazwa']).agg('sum').reset_index()
            Goals2 = StatsForDash[["Zespol2-Nazwa", "Zespol1-Bramki"]].groupby(['Zespol2-Nazwa']).agg('sum').reset_index()
            Goals2.columns = ["Zespol1-Nazwa", "Zespol2-Bramki"]
            Goals1 = Goals1.append(Goals2, ignore_index=True)
            Goals1 = Goals1.groupby(['Zespol1-Nazwa']).agg('sum').reset_index()
            figure = px.histogram(data_frame=Goals1, x = "Zespol1-Nazwa", y = "Zespol2-Bramki").update_xaxes(
                categoryorder='total descending')
        else:
            #Zdobyte
            GoalsZ = StatsForDash[["Zespol1-Nazwa", "Zespol1-Bramki"]].groupby(['Zespol1-Nazwa']).agg('sum').reset_index()
            Goals2 = StatsForDash[["Zespol2-Nazwa", "Zespol2-Bramki"]].groupby(['Zespol2-Nazwa']).agg('sum').reset_index()
            Goals2.columns = ["Zespol1-Nazwa", "Zespol1-Bramki"]
            GoalsZ = GoalsZ.append(Goals2, ignore_index=True)
            GoalsZ = GoalsZ.groupby(['Zespol1-Nazwa']).agg('sum').reset_index()
            
            #Stracone
            GoalsS = StatsForDash[["Zespol1-Nazwa", "Zespol2-Bramki"]].groupby(['Zespol1-Nazwa']).agg('sum').reset_index()
            Goals2 = StatsForDash[["Zespol2-Nazwa", "Zespol1-Bramki"]].groupby(['Zespol2-Nazwa']).agg('sum').reset_index()
            Goals2.columns = ["Zespol1-Nazwa", "Zespol2-Bramki"]
            GoalsS = GoalsS.append(Goals2, ignore_index=True)
            GoalsS = GoalsS.groupby(['Zespol1-Nazwa']).agg('sum').reset_index()
            
            GoalsZ["Zespol1-Bramki"] =  GoalsZ["Zespol1-Bramki"] - GoalsS["Zespol2-Bramki"]
            
            figure = px.histogram(data_frame=GoalsZ, x = "Zespol1-Nazwa", y = "Zespol1-Bramki").update_xaxes(
                categoryorder='total descending')
    return description, figure

            
app.run_server(mode='jupyterlab', port=8052)